In [1]:
import pandas as pd
from pysrc.sampling import theta_adj_reg_data
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

theta_values_iid=pd.read_csv("theta.csv").to_numpy().T
theta_iid=pd.read_csv("beta_theta.csv").to_numpy()
V_samples=pd.read_csv("theta_prior_V_samples.csv").to_numpy()[-50000:,:]
beta_samples=pd.read_csv("theta_prior_beta_samples.csv").to_numpy()[-50000:,:]
eta_samples=pd.read_csv("theta_prior_eta_samples.csv").to_numpy()[-50000:]
nu_samples=pd.read_csv("theta_prior_nu_samples.csv").to_numpy()[-50000:]

In [ ]:
mean_pa_2017=44.97362
theta_fit_df_78= gpd.read_file("/project/lhansen/HMC_re/project-amazon/data/calibration/hmc/theta_fit_78.geojson")
X_fit = theta_adj_reg_data(78,theta_fit_df_78)['X_theta']
weights = theta_adj_reg_data(78,theta_fit_df_78)['G_theta']
fit_ids = theta_fit_df_78["group_id"].values -1   # Column 8 as site_ids
fit_ids = fit_ids.astype(int)
site_theta= np.exp(X_fit @ beta_samples.T + V_samples[:, fit_ids].T)
theta_78 = weights@site_theta/mean_pa_2017
theta_78

In [ ]:


list=['intercept','latitude','latitude^2','temperature','temperature^2','precipitation','distance','farm gate price']

for i in range(8):
    plt.figure(figsize=(12, 6))
    sns.kdeplot(theta_iid[:, i], label='iid', color='blue', fill=True, alpha=0.6,linewidth=4) 
    sns.kdeplot(beta_samples[:, i], label='random effect', color='red', fill=True, alpha=0.6,linewidth=4)  
    plt.title(f'{list[i]}')
    plt.legend()
    plt.savefig('results/theta/beta'+str(i+1)+'.png')
    plt.show()

In [ ]:
# Plotting the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.hist(eta_samples, bins=100, density=True, alpha=0.75)
plt.title(r'Posterior of $\eta$')
plt.subplot(1, 2, 2)
plt.hist(nu_samples, bins=100, density=True, alpha=0.75)
plt.title(r'Posterior of $\nu$')
plt.tight_layout()
plt.savefig('results/theta/precision.png')
plt.show()

In [ ]:
for i in range(62):
    plt.figure(figsize=(12, 6))
    sns.kdeplot(V_samples[:, i], label='random effect', color='blue', fill=True, alpha=0.6,linewidth=4) 
    plt.title(r'$V_{'+str(i+1)+'}$')
    plt.legend()
    plt.savefig('results/theta/V'+str(i+1)+'.png')
    plt.close()

In [ ]:
from numpy import percentile

for i in range(78):
    
    lower_re, upper_re = np.percentile(theta_78[i, :], [0.5, 99.5])
    theta_re_filtered = theta_78[i, (theta_78[i, :] >= lower_re) & (theta_78[i, :] <= upper_re)]

    plt.figure(figsize=(12, 6))
    sns.kdeplot(theta_values_iid[i, :], label='iid', color='blue', fill=True, alpha=0.6,linewidth=4) 
    sns.kdeplot(theta_re_filtered, label='random effect', color='red', fill=True, alpha=0.6,linewidth=4)  
    plt.title(r'$\theta_{'+str(i+1)+'}$')
    plt.legend()
    plt.savefig('results/theta/theta'+str(i+1)+'.png')
    plt.show()

In [3]:
from numpy import percentile

for i in range(78):
    
    lower_re, upper_re = np.percentile(theta_78[i, :], [0.5, 99.5])
    theta_re_filtered = theta_78[i, (theta_78[i, :] >= lower_re) & (theta_78[i, :] <= upper_re)]

    plt.figure(figsize=(12, 6))
    # sns.kdeplot(theta_values_iid[i, :], label='iid', color='blue', fill=True, alpha=0.6,linewidth=4) 
    sns.kdeplot(theta_re_filtered, label='random effect', color='red', fill=True, alpha=0.6,linewidth=4)  
    plt.title(r'$\theta_{'+str(i+1)+'}$')
    plt.legend()
    plt.show()